### Dodatokowy eksperyment

W zadaniu 4 działamy na tym samym zbiorze danych co w zadaniu 3 i można dostrzec następujące zależności:
- w zadaniu 3 dokonujemy transformaci liniowej PCA, natomiast później klasyfikujemy dane m.in. modelem SVM z wybranym jądrem
- w zadaniu 4 posiłkujemy się artykułem, w którym to autorzy przedstawiają wplyw użycia kPCA na reprezentację danych i podział na klasy klasyfikatorem liniowym

Ponieważ w modelu SVM oraz metodzie kPCA można używać tego samego rodzaju jąder, chciałbym porównać wyniki klasyfikacji zdjęć twarzy na podstawie dwóch podejść:
- właściwym dla ćwiczenia 3 użyciu liniowego PCA do przygotowania danych, a następnie ich klasyfikacji SVM z jądrem wielomianowym i RBF
- odpowiadającym ćwiczeniu 4 przygotowaniu danych za pomocą kPCA z jądrem wielomianowym i RBF, a następnie klasyfikacją modelem klasyfikacyjnym z liniowym jądrem

Podejrzewam, że uzyskamy odpowiadające sobie wyniki, gdyż różnica pomiędzy kPCA i SVM używającego tego samego rodzaju jądra jest tylko taka że kPCA przetwarza wszystkie dane podane jednocześnie, natomiast SVM mapuje wartości ad-hoc stosując "kernel trick".
W związku z powyższym porównam efekty klasyfikacji w następujących parach:
- (PCA -> SVM Poly) vs (kPCA Poly -> SVM Linear)
- (PCA -> SVM RBF) vs (kPCA RBF -> SVM Linear)

In [18]:
from sklearn.datasets import fetch_lfw_people
from sklearn.model_selection import train_test_split

lfw_people = fetch_lfw_people(min_faces_per_person=50)

x_train, x_test, y_train, y_test = train_test_split(
    lfw_people.data, lfw_people.target, test_size=0.3, random_state=7)

Znów stosuję taki sam "random_state" jak w zadaniach 3 i 4 dla powtarzalności wyników.

In [19]:
from sklearn.decomposition import PCA

pca_100 = PCA(n_components=100)
pca_100.fit(x_train)
x_train_pca_100 = pca_100.transform(x_train)
x_test_pca_100 = pca_100.transform(x_test)

In [20]:
from sklearn.decomposition import KernelPCA

X_kpca_100 = KernelPCA(n_components=100, kernel='poly', degree=5)
X_kpca_100.fit(x_train)
x_train_kpca_100 = X_kpca_100.transform(x_train)
x_test_kpca_100 = X_kpca_100.transform(x_test)

In [21]:
from sklearn.svm import SVC
from sklearn.metrics import precision_recall_fscore_support

svm_clf = SVC(kernel="poly", degree=5, random_state=7)

svm_clf.fit(x_train_pca_100, y_train)
y_predict = svm_clf.predict(x_test_pca_100)

precision, recall, fscore, _ = precision_recall_fscore_support(y_test, y_predict, zero_division=0)
print('#PCA_components=100, SVM classifier, kernel=poly:')
for i in range(len(precision)):
    print(f'class: {i}, precision: {precision[i]}, recall: {recall[i]}, f1 score: {fscore[i]}')

precision_avg, recall_avg, f1_score_avg, _ = precision_recall_fscore_support(y_test, y_predict, average='weighted', zero_division=0)
print(f'avg-values -> precision: {precision[i]}, recall: {recall[i]}, f1 score: {fscore[i]}')

#PCA_components=100, SVM classifier, kernel=poly:
class: 0, precision: 0.0, recall: 0.0, f1 score: 0.0
class: 1, precision: 0.625, recall: 0.09615384615384616, f1 score: 0.16666666666666666
class: 2, precision: 0.25, recall: 0.025, f1 score: 0.045454545454545456
class: 3, precision: 0.36211031175059955, recall: 0.967948717948718, f1 score: 0.5270506108202443
class: 4, precision: 0.25, recall: 0.023809523809523808, f1 score: 0.043478260869565216
class: 5, precision: 0.0, recall: 0.0, f1 score: 0.0
class: 6, precision: 1.0, recall: 0.09090909090909091, f1 score: 0.16666666666666666
class: 7, precision: 0.16666666666666666, recall: 0.0625, f1 score: 0.09090909090909091
class: 8, precision: 0.0, recall: 0.0, f1 score: 0.0
class: 9, precision: 1.0, recall: 0.25, f1 score: 0.4
class: 10, precision: 0.9333333333333333, recall: 0.7777777777777778, f1 score: 0.8484848484848485
class: 11, precision: 0.8333333333333334, recall: 0.09259259259259259, f1 score: 0.16666666666666666
avg-values -> prec

In [22]:
from sklearn.svm import SVC

svm_clf = SVC(kernel="linear", random_state=7)

svm_clf.fit(x_train_kpca_100, y_train)
y_predict = svm_clf.predict(x_test_kpca_100)

precision, recall, fscore, _ = precision_recall_fscore_support(y_test, y_predict, zero_division=0)
print('#kPCA_components=100, kernel=poly, SVM classifier, kernel=linear:')
for i in range(len(precision)):
    print(f'class: {i}, precision: {precision[i]}, recall: {recall[i]}, f1 score: {fscore[i]}')

precision_avg, recall_avg, f1_score_avg, _ = precision_recall_fscore_support(y_test, y_predict, average='weighted', zero_division=0)
print(f'avg-values -> precision: {precision[i]}, recall: {recall[i]}, f1 score: {fscore[i]}')

#kPCA_components=100, kernel=poly, SVM classifier, kernel=linear:
class: 0, precision: 0.0, recall: 0.0, f1 score: 0.0
class: 1, precision: 0.4868421052631579, recall: 0.7115384615384616, f1 score: 0.578125
class: 2, precision: 0.625, recall: 0.125, f1 score: 0.20833333333333334
class: 3, precision: 0.44542772861356933, recall: 0.967948717948718, f1 score: 0.6101010101010101
class: 4, precision: 0.0, recall: 0.0, f1 score: 0.0
class: 5, precision: 0.0, recall: 0.0, f1 score: 0.0
class: 6, precision: 0.0, recall: 0.0, f1 score: 0.0
class: 7, precision: 0.6666666666666666, recall: 0.125, f1 score: 0.21052631578947367
class: 8, precision: 0.0, recall: 0.0, f1 score: 0.0
class: 9, precision: 0.0, recall: 0.0, f1 score: 0.0
class: 10, precision: 0.8823529411764706, recall: 0.8333333333333334, f1 score: 0.8571428571428571
class: 11, precision: 0.72, recall: 0.3333333333333333, f1 score: 0.45569620253164556
avg-values -> precision: 0.72, recall: 0.3333333333333333, f1 score: 0.455696202531645

Wynik eksperymentu dla 100 komponentów wiodących i jądra wielomianowego o stopniu 5 wyszedł negatywnie.  

Na tych samych danych kombinacja PCA -> SVM(poly) daje dużo gorszą precyzję klasyfikacji, niż kPCA(poly) -> SVM(linear), a uśrednione własności precyzji różnią się i to znacznie. Przypadek z liniowym PCA i SVM(poly) boryka się z bardzo słabą wartością "recall", jednak jest to przypadłóść, którą zaobserwowałem dla tego zbioru przy użyciu SVM(poly).  

Zobaczmy w kolejnym punkcie, jak wypadnie ten sam eksperyment, ale z użyciem jądra RBF.

In [23]:
from sklearn.decomposition import KernelPCA

sigma = 5
gamma = 1.0 / (2 * sigma**2)

X_kpca_100 = KernelPCA(n_components=100, kernel='rbf', gamma=gamma)
X_kpca_100.fit(x_train)
x_train_kpca_100 = X_kpca_100.transform(x_train)
x_test_kpca_100 = X_kpca_100.transform(x_test)

In [24]:
from sklearn.svm import SVC
from sklearn.metrics import precision_recall_fscore_support

svm_clf = SVC(kernel="rbf", gamma=gamma, random_state=7)

svm_clf.fit(x_train_pca_100, y_train)
y_predict = svm_clf.predict(x_test_pca_100)

precision, recall, fscore, _ = precision_recall_fscore_support(y_test, y_predict, zero_division=0)
print('#PCA_components=100, SVM classifier, kernel=rbf:')
for i in range(len(precision)):
    print(f'class: {i}, precision: {precision[i]}, recall: {recall[i]}, f1 score: {fscore[i]}')

precision_avg, recall_avg, f1_score_avg, _ = precision_recall_fscore_support(y_test, y_predict, average='weighted', zero_division=0)
print(f'avg-values -> precision: {precision[i]}, recall: {recall[i]}, f1 score: {fscore[i]}')

#PCA_components=100, SVM classifier, kernel=rbf:
class: 0, precision: 1.0, recall: 0.5555555555555556, f1 score: 0.7142857142857143
class: 1, precision: 0.6615384615384615, recall: 0.8269230769230769, f1 score: 0.7350427350427351
class: 2, precision: 0.7916666666666666, recall: 0.475, f1 score: 0.59375
class: 3, precision: 0.569811320754717, recall: 0.967948717948718, f1 score: 0.7173396674584323
class: 4, precision: 0.7777777777777778, recall: 0.3333333333333333, f1 score: 0.4666666666666667
class: 5, precision: 1.0, recall: 0.36, f1 score: 0.5294117647058824
class: 6, precision: 1.0, recall: 0.09090909090909091, f1 score: 0.16666666666666666
class: 7, precision: 1.0, recall: 0.375, f1 score: 0.5454545454545454
class: 8, precision: 1.0, recall: 0.125, f1 score: 0.2222222222222222
class: 9, precision: 0.9285714285714286, recall: 0.65, f1 score: 0.7647058823529411
class: 10, precision: 1.0, recall: 0.8333333333333334, f1 score: 0.9090909090909091
class: 11, precision: 0.7435897435897436

In [25]:
from sklearn.svm import SVC

svm_clf = SVC(kernel="linear", random_state=7)

svm_clf.fit(x_train_kpca_100, y_train)
y_predict = svm_clf.predict(x_test_kpca_100)

precision, recall, fscore, _ = precision_recall_fscore_support(y_test, y_predict, zero_division=0)
print('#kPCA_components=100, kernel=rbf, SVM classifier, kernel=linear:')
for i in range(len(precision)):
    print(f'class: {i}, precision: {precision[i]}, recall: {recall[i]}, f1 score: {fscore[i]}')

precision_avg, recall_avg, f1_score_avg, _ = precision_recall_fscore_support(y_test, y_predict, average='weighted', zero_division=0)
print(f'avg-values -> precision: {precision[i]}, recall: {recall[i]}, f1 score: {fscore[i]}')

#kPCA_components=100, kernel=rbf, SVM classifier, kernel=linear:
class: 0, precision: 1.0, recall: 0.05555555555555555, f1 score: 0.10526315789473684
class: 1, precision: 0.5522388059701493, recall: 0.7115384615384616, f1 score: 0.6218487394957983
class: 2, precision: 0.6666666666666666, recall: 0.15, f1 score: 0.24489795918367346
class: 3, precision: 0.4353932584269663, recall: 0.9935897435897436, f1 score: 0.60546875
class: 4, precision: 0.0, recall: 0.0, f1 score: 0.0
class: 5, precision: 0.0, recall: 0.0, f1 score: 0.0
class: 6, precision: 0.0, recall: 0.0, f1 score: 0.0
class: 7, precision: 1.0, recall: 0.1875, f1 score: 0.3157894736842105
class: 8, precision: 0.0, recall: 0.0, f1 score: 0.0
class: 9, precision: 1.0, recall: 0.3, f1 score: 0.46153846153846156
class: 10, precision: 1.0, recall: 0.6111111111111112, f1 score: 0.7586206896551724
class: 11, precision: 0.6666666666666666, recall: 0.18518518518518517, f1 score: 0.2898550724637681
avg-values -> precision: 0.66666666666666


Niestety wynik eksperymentu okazał się negatywny.  

I w tym przypadku, pomiary precyzji klasyfikacji dla tego samego zestawu danych wypada kompletnie inaczej dla przetwarzania PCA -> SVM(rbf), oraz kPCA(rbf) -> SVM(linear). Tym razem, to kombinacja PCA -> SVM(rbf) daje lepszą precyzję a sprawą dużo lepszego "recall" i stan ten nie zmienia się po dopasowaniu współczynnika gamma. 
